# SpaceX Data Analysis with SQL

This notebook conducts SQL-based analysis on SpaceX launch data stored in a SQLite database. It loads data from a CSV file into a SQL table and executes various queries to analyze different aspects of SpaceX launches.

Key components:

*   Data loading into SQLite database
*   Custom function for displaying query results
*   SQL queries ranging from basic to complex analyses
*   Structured sections: setup, data preparation, and progressive SQL analyses

## 1. Setup

In [1]:
import csv, sqlite3
import pandas as pd
%load_ext sql

# Constants
DATA_URL = "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_2/data/Spacex.csv"

## 2. Helper Function

## 2. Data Loading

In [2]:
# Create a SQLite database connection
conn = sqlite3.connect("spacex_data.db")

# Connect to SQLite database
%sql sqlite:///spacex_data.db

# Load data from CSV and create table, removing rows with null dates
df = pd.read_csv(DATA_URL)
df_cleaned = df[df['Date'].notnull()]
df_cleaned.to_sql("SPACEX_TABLE", conn, if_exists='replace', index=False, method="multi")

# Verify data was loaded
result = %sql SELECT COUNT(*) FROM SPACEX_TABLE
print(f"Number of rows in SPACEX_TABLE: {result[0][0]}")

# Close connection
conn.close()

 * sqlite:///spacex_data.db
Done.
Number of rows in SPACEX_TABLE: 101


## 3. SQL Queries and Analysis

### Task 1: Display unique launch sites

In [3]:
%sql SELECT DISTINCT LAUNCH_SITE FROM SPACEX_TABLE;

 * sqlite:///spacex_data.db
Done.


Launch_Site
CCAFS LC-40
VAFB SLC-4E
KSC LC-39A
CCAFS SLC-40


### Task 2: Display 5 records where launch sites begin with 'CCA'

In [4]:
%sql SELECT * FROM SPACEX_TABLE WHERE LAUNCH_SITE LIKE 'CCA%' LIMIT 5;

 * sqlite:///spacex_data.db
Done.


Date,Time (UTC),Booster_Version,Launch_Site,Payload,PAYLOAD_MASS__KG_,Orbit,Customer,Mission_Outcome,Landing_Outcome
2010-06-04,18:45:00,F9 v1.0 B0003,CCAFS LC-40,Dragon Spacecraft Qualification Unit,0,LEO,SpaceX,Success,Failure (parachute)
2010-12-08,15:43:00,F9 v1.0 B0004,CCAFS LC-40,"Dragon demo flight C1, two CubeSats, barrel of Brouere cheese",0,LEO (ISS),NASA (COTS) NRO,Success,Failure (parachute)
2012-05-22,7:44:00,F9 v1.0 B0005,CCAFS LC-40,Dragon demo flight C2,525,LEO (ISS),NASA (COTS),Success,No attempt
2012-10-08,0:35:00,F9 v1.0 B0006,CCAFS LC-40,SpaceX CRS-1,500,LEO (ISS),NASA (CRS),Success,No attempt
2013-03-01,15:10:00,F9 v1.0 B0007,CCAFS LC-40,SpaceX CRS-2,677,LEO (ISS),NASA (CRS),Success,No attempt


### Task 3: Display total payload mass carried by NASA (CRS) boosters

In [5]:
%sql SELECT SUM(PAYLOAD_MASS__KG_) AS payloadmass FROM SPACEX_TABLE WHERE Customer = 'NASA (CRS)';

 * sqlite:///spacex_data.db
Done.


payloadmass
45596


### Task 4: Display average payload mass carried by F9 v1.1 boosters

In [6]:
%sql SELECT ROUND(AVG(PAYLOAD_MASS__KG_), 2) AS avg_payload_mass FROM SPACEX_TABLE WHERE Booster_Version = 'F9 v1.1';

 * sqlite:///spacex_data.db
Done.


avg_payload_mass
2928.4


### Task 5: Date of first successful ground pad landing

In [7]:
%sql SELECT MIN(Date) FROM SPACEX_TABLE WHERE "Landing_Outcome" = 'Success (ground pad)';

 * sqlite:///spacex_data.db
Done.


MIN(Date)
2015-12-22


## 4. Advanced Queries

### Task 6: Boosters with successful drone ship landings and payload mass between 4000-6000 kg

In [8]:
%sql SELECT BOOSTER_VERSION FROM SPACEX_TABLE WHERE LANDING_OUTCOME='Success (drone ship)' AND PAYLOAD_MASS__KG_ BETWEEN 4000 AND 6000;

 * sqlite:///spacex_data.db
Done.


Booster_Version
F9 FT B1022
F9 FT B1026
F9 FT B1021.2
F9 FT B1031.2


### Task 7: Total number of successful and failed mission outcomes

In [9]:
%sql SELECT MISSION_OUTCOME, COUNT(*) AS missionoutcomes FROM SPACEX_TABLE GROUP BY MISSION_OUTCOME;

 * sqlite:///spacex_data.db
Done.


Mission_Outcome,missionoutcomes
Failure (in flight),1
Success,98
Success,1
Success (payload status unclear),1


### Task 8: Booster versions with maximum payload mass

In [10]:
%sql SELECT BOOSTER_VERSION FROM SPACEX_TABLE WHERE PAYLOAD_MASS__KG_ = (SELECT MAX(PAYLOAD_MASS__KG_) FROM SPACEX_TABLE);

 * sqlite:///spacex_data.db
Done.


Booster_Version
F9 B5 B1048.4
F9 B5 B1049.4
F9 B5 B1051.3
F9 B5 B1056.4
F9 B5 B1048.5
F9 B5 B1051.4
F9 B5 B1049.5
F9 B5 B1060.2
F9 B5 B1058.3
F9 B5 B1051.6


### Task 9: Records for 2015 showing month, failed drone ship landings, booster versions, and launch sites

In [11]:
%sql SELECT SUBSTR(Date, 6, 2) AS Month, "Landing_Outcome", Booster_Version, Launch_Site FROM SPACEX_TABLE WHERE SUBSTR(Date, 1, 4) = '2015' AND "Landing_Outcome" = 'Failure (drone ship)';

 * sqlite:///spacex_data.db
Done.


Month,Landing_Outcome,Booster_Version,Launch_Site
01,Failure (drone ship),F9 v1.1 B1012,CCAFS LC-40
04,Failure (drone ship),F9 v1.1 B1015,CCAFS LC-40


### Task 10: Ranked landing outcomes between 2010-06-04 and 2017-03-20

In [12]:
%sql SELECT LANDING_OUTCOME, COUNT(*) AS Count FROM SPACEX_TABLE WHERE DATE BETWEEN '2010-06-04' AND '2017-03-20' GROUP BY LANDING_OUTCOME ORDER BY Count DESC;

 * sqlite:///spacex_data.db
Done.


Landing_Outcome,Count
No attempt,10
Success (drone ship),5
Failure (drone ship),5
Success (ground pad),3
Controlled (ocean),3
Uncontrolled (ocean),2
Failure (parachute),2
Precluded (drone ship),1
